In [1]:
import cv2
import os
import glob
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd

import random

# Deep Learning
import keras
import tensorflow as tf
from keras.models import Sequential, Model
from keras.layers import Dense,  SeparableConv2D, Conv2D , Conv3D, MaxPool3D, MaxPool2D , Flatten, Input, Concatenate, concatenate , Dropout , BatchNormalization, Activation
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix
from keras.callbacks import ReduceLROnPlateau
import cv2
from keras.preprocessing.image import ImageDataGenerator

In [6]:
from bibliotheque_projet import import_data, separation_data, creation_model, fit_model, creation_y

In [7]:
labels = ["1" , "2" , "3" , "4"]
electrodes = ["C3" , "C4" , "Cz"]

racine_ondelettes = "/content/drive/MyDrive/Projet_E4_Interface_Cerveau_Machine/Bases de données EEG/Bases de données/bdd_ondelettes"

In [9]:
data_scalo = import_data(electrodes[0], racine_ondelettes, labels)
X_train1, X_test1, X_val1, y_train1, y_test1, y_val1 = separation_data(data_scalo)
print(X_train1.shape)

data_scalo2 = import_data(electrodes[1], racine_ondelettes, labels)
X_train2, X_test2, X_val2, y_train2, y_test2, y_val2 = separation_data(data_scalo)
print(X_train2.shape)

data_scalo3 = import_data(electrodes[2], racine_ondelettes, labels)
X_train3, X_test3, X_val3, y_train3, y_test3, y_val3 = separation_data(data_scalo)
print(X_train3.shape)

(326, 250, 250, 3)
(326, 250, 250, 3)
(326, 250, 250, 3)


In [10]:
y_train = creation_y(y_train1)
y_test = creation_y(y_test1)
y_val = creation_y(y_val1)

In [13]:
np.argmax(y_test, axis = 1)

array([3, 0, 0, 1, 1, 3, 0, 1, 0, 3, 0, 0, 3, 3, 1, 2, 1, 1, 3, 1, 2, 1,
       0, 1, 0, 1, 2, 2, 1, 3, 3, 2, 0, 2, 1, 1, 3, 0, 0, 0, 3, 1, 3, 2,
       0, 2, 0, 2, 2, 3, 0, 3, 0, 3, 3, 0, 1, 2, 1, 0, 2, 2, 2, 3, 0, 2,
       1, 1, 3, 3, 2, 1, 3, 1, 2, 0, 3])

#Modèle 1

In [40]:
def importation_entrainement() :
  X_train = X_train1
  y_train = y_train1
  X_val = X_val1
  y_val = y_val1
  model = creation_model(np.shape(X_train))
  model.compile(loss= "binary_crossentropy", optimizer="adam", metrics=["accuracy"])
  epochs = 5
  batch_size = int(len(X_train)/ 6)
  steps = int(len(X_train) / 32)
  history = fit_model(model, epochs, batch_size, steps, X_train, X_val, y_train, y_val)
  return history

In [9]:
test_1 = importation_entrainement()

Epoch 1/5
10/10 [==============================] - 36s 375ms/step - loss: -22.4756 - accuracy: 0.2814 - val_loss: -259.8080 - val_accuracy: 0.0000e+00
Epoch 2/5
10/10 [==============================] - 3s 334ms/step - loss: -382.1208 - accuracy: 0.2806 - val_loss: -2621.3516 - val_accuracy: 0.0000e+00
Epoch 3/5
10/10 [==============================] - 3s 331ms/step - loss: -2540.6136 - accuracy: 0.3333 - val_loss: -16547.0195 - val_accuracy: 0.0000e+00
Epoch 4/5
10/10 [==============================] - 3s 332ms/step - loss: -13492.1058 - accuracy: 0.3137 - val_loss: -73155.5781 - val_accuracy: 0.0000e+00
Epoch 5/5
10/10 [==============================] - 3s 333ms/step - loss: -51667.8136 - accuracy: 0.3078 - val_loss: -264781.2188 - val_accuracy: 0.0000e+00


#Modele 2

In [109]:
shape = np.shape(X_train1)

input_1 = Input(shape=shape[1:])
input_2 = Input(shape=shape[1:])
input_3 = Input(shape=shape[1:])

model = concatenate([input_1, input_2, input_3], axis = -1)

model = Conv2D(16, (3,3)) (model)
model = Activation("relu") (model)
model = MaxPool2D(16, (2,2)) (model)
model = Activation('relu') (model)
#model = Dense(16) (model)
model = Dropout(0.2) (model)

model = Conv2D(32, (3,3)) (model)
model = Activation("relu") (model)
model = MaxPool2D(32, (2,2)) (model)
model = Activation('relu') (model)
#model = Dense(32) (model)
model = Dropout(0.2) (model)

model = Flatten() (model)
model = Dense(64) (model)
model = Activation('relu') (model)
#model = Dropout(0.2) (model)
model = Dense(4) (model)

output = Activation('softmax') (model)

model = Model([input_1, input_2, input_3], output)

model.summary()

Model: "model_26"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_112 (InputLayer)          [(None, 250, 250, 3) 0                                            
__________________________________________________________________________________________________
input_113 (InputLayer)          [(None, 250, 250, 3) 0                                            
__________________________________________________________________________________________________
input_114 (InputLayer)          [(None, 250, 250, 3) 0                                            
__________________________________________________________________________________________________
concatenate_37 (Concatenate)    (None, 250, 250, 9)  0           input_112[0][0]                  
                                                                 input_113[0][0]           

In [110]:
epochs = 75
batch_size = int(len(X_train1)/ 12)

datagen = ImageDataGenerator( shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
datagen.fit(X_train1)

model.compile(loss= "categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit([X_train1,X_train2,X_train3], y_train, batch_size = 32, 
          steps_per_epoch= int(len(X_train1) / 32),
          epochs=epochs,
          validation_data = ([X_val1, X_val2,X_val3], y_val))

Epoch 1/75
10/10 [==============================] - 2s 115ms/step - loss: 23.3381 - accuracy: 0.2382 - val_loss: 1.3845 - val_accuracy: 0.4000
Epoch 2/75
10/10 [==============================] - 1s 90ms/step - loss: 1.3866 - accuracy: 0.2182 - val_loss: 1.3672 - val_accuracy: 0.4000
Epoch 3/75
10/10 [==============================] - 1s 84ms/step - loss: 1.4270 - accuracy: 0.3170 - val_loss: 1.3833 - val_accuracy: 0.4000
Epoch 4/75
10/10 [==============================] - 1s 84ms/step - loss: 1.3861 - accuracy: 0.3047 - val_loss: 1.3832 - val_accuracy: 0.4000
Epoch 5/75
10/10 [==============================] - 1s 81ms/step - loss: 1.3865 - accuracy: 0.2342 - val_loss: 1.3834 - val_accuracy: 0.4000
Epoch 6/75
10/10 [==============================] - 1s 83ms/step - loss: 1.3858 - accuracy: 0.2924 - val_loss: 1.3833 - val_accuracy: 0.4000
Epoch 7/75
10/10 [==============================] - 1s 84ms/step - loss: 1.3860 - accuracy: 0.2807 - val_loss: 1.3834 - val_accuracy: 0.4000
Epoch 8/75


In [111]:
y_pred = model.predict([X_test1,X_test2,X_test3])

In [112]:
np.argmax(y_pred, axis = 1)

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [113]:
np.argmax(y_test, axis = 1)

array([3, 0, 0, 1, 1, 3, 0, 1, 0, 3, 0, 0, 3, 3, 1, 2, 1, 1, 3, 1, 2, 1,
       0, 1, 0, 1, 2, 2, 1, 3, 3, 2, 0, 2, 1, 1, 3, 0, 0, 0, 3, 1, 3, 2,
       0, 2, 0, 2, 2, 3, 0, 3, 0, 3, 3, 0, 1, 2, 1, 0, 2, 2, 2, 3, 0, 2,
       1, 1, 3, 3, 2, 1, 3, 1, 2, 0, 3])

In [114]:
res = np.argmax(y_pred, axis = 1) -np.argmax(y_test, axis = 1)

In [115]:
(res == 0).sum()/len(res)

0.22077922077922077

#Modele 3

In [125]:
shape = (250,250,3)

input_1 = Input(shape=shape[1:])
input_2 = Input(shape=shape[1:])
input_3 = Input(shape=shape[1:])
input_4 = Input(shape=shape[1:])
input_5 = Input(shape=shape[1:])
input_6 = Input(shape=shape[1:])

#modele 1
model_1 = Sequential()
model_1.add(Conv2D(32, (3,3), input_dim = shape))
model_1.add(Activation("relu"))
model_1.add(MaxPool2D(32, (2,2)))
model_1.add(Activation('relu'))
model_1.add(Dense(32))
model_1.add(Dropout(0.2))

model_1.add(Conv2D(64, (3,3)))
model_1.add(Activation("relu"))
model_1.add(MaxPool2D(64, (2,2)))
model_1.add(Activation('relu'))
model_1.add(Dense(64))
model_1.add(Dropout(0.25))

model_1.add(Flatten())
model_1.add(Dense(64))
model_1.add(Activation('relu'))
model_1.add(Dropout(0.5))
model_1.add(Dense(4))
model_1.add(Activation('softmax'))

#modele2
model_2 = Sequential()
model_2.add(Conv2D(32, (3,3), input_dim = shape))
model_2.add(Activation("relu"))
model_2.add(MaxPool2D(32, (2,2)))
model_2.add(Activation('relu'))
model_2.add(Dense(32))
model_2.add(Dropout(0.2))

model_2.add(Conv2D(64, (3,3)))
model_2.add(Activation("relu"))
model_2.add(MaxPool2D(64, (2,2)))
model_2.add(Activation('relu'))
model_2.add(Dense(64))
model_2.add(Dropout(0.25))

model_2.add(Flatten())
model_2.add(Dense(64))
model_2.add(Activation('relu'))
model_2.add(Dropout(0.5))
model_2.add(Dense(4))
model_2.add(Activation('softmax'))

#modele3
model_3 = Sequential()
model_3.add(Conv2D(32, (3,3), input_dim = shape))
model_3.add(Activation("relu"))
model_3.add(MaxPool2D(32, (2,2)))
model_3.add(Activation('relu'))
model_3.add(Dense(32))
model_3.add(Dropout(0.2))

model_3.add(Conv2D(64, (3,3)))
model_3.add(Activation("relu"))
model_3.add(MaxPool2D(64, (2,2)))
model_3.add(Activation('relu'))
model_3.add(Dense(64))
model_3.add(Dropout(0.25))

model_3.add(Flatten())
model_3.add(Dense(64))
model_3.add(Activation('relu'))
model_3.add(Dropout(0.5))
model_3.add(Dense(4))
model_3.add(Activation('softmax'))

TypeError: ignored

In [117]:
epochs = 50
batch_size = int(len(X_train1)/ 12)

datagen = ImageDataGenerator( shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
datagen.fit(X_train1)

model_1.compile(loss= "categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model_1.fit(datagen.flow([X_train1, X_train2], y_train, batch_size = 32), 
          steps_per_epoch= int(len(X_train1) / 32),
          epochs=epochs,
          validation_data = ([X_val1, X_val2], y_val))

Epoch 1/50
10/10 [==============================] - 5s 425ms/step - loss: 6.0567 - accuracy: 0.1760 - val_loss: 7.4473 - val_accuracy: 0.2000
Epoch 2/50
 2/10 [=====>........................] - ETA: 3s - loss: 1.5866 - accuracy: 0.2734

KeyboardInterrupt: ignored